In [ ]:
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt
import matplotlib
from sobol_lib import *
from mpl_toolkits.mplot3d import Axes3D
from gen_mat import *
from block_rect_maxvol import *
import re
import os
from matplotlib import cm
from mva_test import *
import itertools
from ipywidgets import interactive, interact, widgets
#%matplotlib inline
%matplotlib notebook
import matplotlib.pyplot as plt
from maxvolpy.maxvol import rect_maxvol
os.environ['OMP_NUM_THREADS'] = '6'
print (os.environ['OMP_NUM_THREADS'])

In [ ]:
num_expr = 1 # Adding this num to output fn for distinguish between experiments  (not used for now)
nder = 2 # Dimension
ToLoadSet = False # whether to load settigns form file

cut_radiuses = [0.09]
inital_points_distribs = ['LHS']

if ToLoadSet:
    from sandbox_bm_approx_settings import *
else:
    dir_str = './cr_test_electron'
    #dir_str = os.path.join(os.environ['HOME'], 'work/res/bm-big/')
    num_points_for_big_matrix = 30000 # number of points for big matrix
    max_row =  70                   # Maximum number of points taken in experiments
    max_expansion = 30         # number of columns in big matrix in (nder+1) units
    min_expansion = 10        # minimal number of columns in experiments in (nder+1) units
    

    n_test = 120000    # points on test grid (for calculating error on final step)
    poly = cheb       # used polinomials
    complex_domain = False
    domain_type = None
    
    if domain_type != None:
           complex_domain = True


#####
dir_pdf = os.path.join(dir_str, "pdf")
try:
    os.makedirs(dir_pdf)
except:
    pass



ToTakePointsFromFile = False # (not used for now)

# ---------------------------------
p_size = (nder+1)*max_row #number of rows in big matrix

### generating test points
points_test = test_points_gen(n_test, nder, distrib='random')
if complex_domain:
    points_test = complex_area_pnts_gen(n_test, nder, distrib='LHS', mod = domain_type)

#contents of file sandbox_bm_approx_settings.py
_="""
from gen_mat import cheb


dir_str = '.'
num_points_for_big_matrix = 50000 # number of points for big matrix
max_row = 50                      # Maximum number of points taken in experiments
max_expansion = 10                 # number of columns in big matrix in (nder+1) units
min_expansion = 1                 # minimal number of columns in experiments in (nder+1) units

n_test = 50000    # points on test grid (for calculating error on final step)
poly = cheb       # used polynomials
"""

### Testing the new most general environment

In [ ]:
### evaluating test
for inital_points_distrib in inital_points_distribs:
    points_fn = 'taken_points_{}'.format(inital_points_distrib)
    x = test_points_gen(num_points_for_big_matrix, nder, distrib=inital_points_distrib)
    if complex_domain:
        x = domain_erase(x, mod = domain_type)
    A = GenMat(p_size, x, poly=poly, debug=False, pow_p=1)
    A = matrix_prep(A, nder+1)
    
    np.savez(os.path.join(dir_str, points_fn), x=x, points_test=points_test)
    
    
    
    fn_pre_pdf = "distrib={}".format(inital_points_distrib)
    for cut_radius in cut_radiuses:
        f = open(os.path.join(dir_str, "distrib={}_radius={}".format(inital_points_distrib, cut_radius) + '.txt'), "w")
        for expansion in range(min_expansion, max_expansion):
                    for N_rows_ex in range(max_row, expansion, -1): # It's not the way people do...
                        N_rows = N_rows_ex*(nder+1)
                        fnpdf = os.path.join(dir_pdf, fn_pre_pdf + "_expansion={}_N_rows_ex={}.pdf".format(expansion, N_rows_ex))
                        try:
                            taken_points = test_bm(A, x,nder, expansion, N_rows, cut_radius = cut_radius,to_save_pivs=N_rows_ex==max_row, 
                                                       fnpdf=fnpdf)
                        except SingularError as err:
                            print ('not full column rank with expansion={}, N_rows_ex={}, err={}'.format(
                                                                expansion, N_rows_ex, err.value)) 
                            #continue
                            break


                        taken_points.tofile(f, sep=" ")
                        f.write("_Nrows={}_expans={}\n".format(N_rows, expansion))
                        f.flush()


        f.close()


In [ ]:
def file_extraction(Filepath, new_extr = True):
    if new_extr:
        srch = re.compile(r'([\d\s]+)_Nrows=(\d+)_expans=(\d+)')
        fnd = srch.findall(open(Filepath, 'r').read())
        return tuple(np.array(i) for i in zip(*[(int(i1), int(i2), [int(p) for p in im1.strip().split(' ') if len(p) > 0])
                                                for im1, i1, i2 in fnd]))
    else:
        srch = re.compile(r'([\d\s]+)_error=([\+\-\d\.eE]+)_Nrows=(\d+)_expans=(\d+)')
        fnd = srch.findall(open(Filepath, 'r').read())
        return tuple(np.array(i) for i in zip(*[(float(i0), int(i1), int(i2), [int(p) for p in im1.strip().split(' ') if len(p) > 0])
                                                for im1, i0, i1, i2 in fnd]))
def DataToMesh(error, N_row, N_col, *args):
    row_s = sorted(list(set(N_row)))
    col_s = sorted(list(set(N_col)))
    data = {(N_row[i], N_col[i]) : e for i, e in enumerate(error)}
    
    res = np.empty((len(row_s), len(col_s)), dtype=float)
    for i, r in enumerate(row_s):
        for j, c in enumerate(col_s):
            try:
                res[i,j] = data[(r, c)]
            except:
                res[i,j] = np.nan
    X, Y = np.meshgrid(row_s, col_s)
    return res.T, X, Y

def PlotError_3D(N_row, N_col, error_ext, log_it=False):
    error, N_row, N_col = DataToMesh(error_ext, N_row, N_col)
    
    if log_it:
        error = np.log10(error)
    
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    ax.plot_surface(N_row, N_col, error, edgecolor='black', linewidth=0.5, cmap = cm.Spectral)
    # ax.legend()
    ax.set_xlabel('N_rows', fontsize=10)
    ax.set_ylabel('N_columns')
    plt.show()

def GenSobol( N = 200, dim = 2, seed = 0, rng=(-1.0, 1.0) ):
    res = np.zeros((N, dim), dtype=float)
    rng_d = rng[1] - rng[0]
    for i in xrange(N):
        res[i, :], seed = i4_sobol ( dim, seed )
        res[i, :] = res[i, :]*rng_d + rng[0]

    return res

### Experimental data processing

In [ ]:
### Data extraction
dir_str = './'
dir_points = os.path.join(dir_str, "cr_test_electron/")
points_distrib = 'LHS'
rad = "0.09"
taken_points = np.load(dir_points + "taken_points_" + points_distrib + ".npz")
x = taken_points['x']
points_test = taken_points['points_test']

In [ ]:
### .txt processing
functions = [f_roots]
N_row, N_col, p_indices = file_extraction(dir_points + "distrib=" + points_distrib + "_radius=" + rad + ".txt")

In [ ]:
#error = np.empty(N_row.shape)
error_e, Leb_e = [], []
#error_sc, Leb_sc = [],[]
#Leb = np.empty(N_row.shape)

if type(functions) is not list:
        functions = [functions]
#if type(function2) is not list:
#        function2 = [function2]
ValsandNorms = MakeValsAndNorms(functions, points_test)
#ValsandNorms_sc = MakeValsAndNorms(function2, points_test)
for j, p in enumerate(p_indices):

    a,b = LebesgueConst(x[p], N_col[j]*(nder+1), poly=cheb, test_pnts=points_test, pow_p=1, funcs=ValsandNorms, derivative = True)
    #c,d = LebesgueConst(x[p], N_col[j]*(nder+1), poly=cheb, test_pnts=points_test, pow_p=1, funcs=ValsandNorms_sc, derivative = True)
    error_e.append(b)
    Leb_e.append(a)
    #error_sc.append(d)
    #Leb_sc.append(c)

In [ ]:
@jit
def mult_error(N_iter, N_col, p_indices,functions, shape=None):
    error_LHS = np.empty([N_col.shape[0], N_iter])
    error_random = np.empty([N_col.shape[0], N_iter])
    error_sobol = np.empty([N_col.shape[0], N_iter])
    if type(functions) is not list:
            functions = [functions]
    ValsandNorms = MakeValsAndNorms(functions, points_test)
    for i in np.arange(N_iter):
        print(i)
        x_LHS = complex_area_pnts_gen(len(p_indices[0]), nder, mod=shape, distrib='LHS')
        x_random = complex_area_pnts_gen(len(p_indices[0]), nder, mod=shape, distrib='random')
        x_sobol = complex_area_pnts_gen(len(p_indices[0]), nder, mod=shape, distrib='Sobol')
        for j, p in enumerate(p_indices):
            _, error_LHS[j,i] = LebesgueConst(x_LHS, N_col[j]*(nder+1), poly=cheb, test_pnts=points_test, pow_p=1, funcs=ValsandNorms, derivative = True)
            _, error_random[j,i] = LebesgueConst(x_random, N_col[j]*(nder+1), poly=cheb, test_pnts=points_test, pow_p=1, funcs=ValsandNorms, derivative = True)
            _, error_sobol[j,i] = LebesgueConst(x_sobol, N_col[j]*(nder+1), poly=cheb, test_pnts=points_test, pow_p=1, funcs=ValsandNorms, derivative = True)
            if j!= p_indices.shape[0]-1:
                if len(p_indices[j+1])>len(p_indices[j]):
                    x_LHS = complex_area_pnts_gen(len(p_indices[0]), nder, mod=shape, distrib='LHS')
                    x_random = complex_area_pnts_gen(len(p_indices[0]), nder, mod=shape, distrib='random')
                    x_sobol = complex_area_pnts_gen(len(p_indices[0]), nder, mod=shape, distrib='Sobol')
                else:
                    x_LHS = x_LHS[:x_LHS.shape[0]-1]
                    x_random = x_random[:x_random.shape[0]-1]
                    x_sobol = x_sobol[:x_sobol.shape[0]-1]
    return(error_LHS,error_random,error_sobol)

In [ ]:
@jit
def multi_piston_error(N_iter, N_row, N_col, p_indices,functions):
    error_LHS = np.empty([N_row[::10].shape[0], N_iter])
    error_random = np.empty([N_row[::10].shape[0], N_iter])
    error_sobol = np.empty([N_row[::10].shape[0], N_iter])
    if type(functions) is not list:
            functions = [functions]
    for i in np.arange(N_iter):
        print(i)
        x_LHS = test_points_gen(num_points_for_big_matrix, nder, distrib='LHS')
        x_random = test_points_gen(num_points_for_big_matrix, nder, distrib='random')
        x_sobol = GenSobol( N = num_points_for_big_matrix, dim = nder, seed=random.randint(0,256),rng=(-1, 1) )
        for j, p in enumerate(p_indices):
            if (j%10 == 0):
                ValsandNorms = MakeValsAndNorms(functions, points_test)
                _, error_LHS[j/10,i] = LebesgueConst(x_LHS[p], N_col[j]*(nder+1), poly=cheb, test_pnts=points_test, pow_p=1, funcs=ValsandNorms, derivative = True)
                _, error_random[j/10,i] = LebesgueConst(x_random[p], N_col[j]*(nder+1), poly=cheb, test_pnts=points_test, pow_p=1, funcs=ValsandNorms, derivative = True)
                _, error_sobol[j/10,i] = LebesgueConst(x_sobol[p], N_col[j]*(nder+1), poly=cheb, test_pnts=points_test, pow_p=1, funcs=ValsandNorms, derivative = True)
                np.savez(os.path.join("piston_10"),er_lhs = error_LHS,er_r = error_random,er_s = error_sobol)

                return(error_LHS,error_random,error_sobol)

In [ ]:
a,b,c = mult_error(10,N_col,p_indices,functions)

In [ ]:
a,b,c = multi_piston_error(20,N_row,N_col,p_indices,functions)

In [ ]:
PlotError_3D(N_row, N_col, error_e, True)

In [ ]:
x_sobol = complex_area_pnts_gen(15, nder, mod='ellipse', distrib='sobol')

### Random points plot

In [ ]:
def multi_error_slice(fn, k, error, error_LHS, error_random, error_sobol, log_it=False, adjust = False):
    N_row, N_col,_ = file_extraction(fn, new_extr=True)
    ndim = 3
    cols = np.arange(N_col[0], N_col[-1])
    fig = plt.figure(figsize=(10,6)) 
    to_floor = True
    if log_it:
            error = np.log10(error)
            error_LHS = np.log10(error_LHS)
            error_random = np.log10(error_random)
            error_sobol = np.log10(error_sobol)
    if adjust == False:
        indx = [] 
        for i in cols:
            if i == 5:
                continue
            ix = np.where(N_col == i)[0]
            if to_floor:
                loc_indx = np.where(N_row[ix]/ndim == np.floor(k*i))[0]
            else:
                loc_indx = np.where(N_row[ix]/ndim == np.ceil(k*i))[0]

            try:### to get absolute index - make a summation 
                indx.append(loc_indx[0] + ix[0])
            except:
                break
    else:
        indx = []
        for i in cols:
            if i == 5:
                continue
            ix = np.where(N_col == i)[0]    
            loc_indx = np.where(N_row[ix] == (i + k)*ndim)[0]

            try:
                indx.append(loc_indx[0] + ix[0])
            except:
                continue
    N_col = 3*N_col            
    plt.plot(N_col[indx], error[indx], label = "BMaxvol", linewidth = 3)
    plt.plot(N_col[indx], error_LHS[indx], label = "LHS")
    plt.plot(N_col[indx], error_random[indx], label = "Random")
    plt.plot(N_col[indx], error_sobol[indx], label = "Sobol")
    plt.legend()
    #fig.suptitle('Approximation error with N_rows = {} N_monoms'.format(k), fontsize=20)
    plt.xlabel('Number of monoms', fontsize=10)
    plt.ylabel('Logscaled error')
    plt.grid(True)
    fnpdf = 'err(monoms)_slice={}.pdf'.format(k)
    plt.grid(True)
    plt.savefig(fnpdf)
    plt.close(fig)
    
    
def multi_error_straight(fn, error, err_LHS, err_rand, err_sob, col, rows, confidence = False):
    ndim = 3
    k = 0.403
    error = np.array(error)
    N_row, N_col,_ = file_extraction(fn, new_extr=True)
    error_LHS = np.mean(err_LHS, axis = 1) 
    error_random = np.mean(err_rand,axis = 1)
    error_sobol = np.mean(err_sob,axis = 1)
    if confidence:
        error_lhs_up = k*np.std(err_LHS,axis=1) + error_LHS
        error_lhs_down = error_LHS - k*np.std(err_LHS,axis=1)

        error_rand_up = k*np.std(err_rand,axis=1) + error_random
        error_rand_down = error_random - k*np.std(err_rand,axis=1)

        error_sob_up = k*np.std(err_sob,axis=1) + error_sobol
        error_sob_down = error_sobol - k*np.std(err_sob,axis=1)
  
    if col is not None:
        cols = np.arange(N_col[0], N_col[-1])
        #if type(cols) is not list:
            #cols = [cols]
        for i in cols:
            indx = np.where(N_col == i)          
            fig = plt.figure()
            plt.plot(N_row[indx]/ndim, error[indx], label = 'BMaxvol', linewidth= 3)
            plt.plot(N_row[indx]/ndim, error_LHS[indx], label = "LHS",color = '#539caf')
            plt.plot(N_row[indx]/ndim, error_random[indx], label = "Random",color = 'green')
            plt.plot(N_row[indx]/ndim, error_sobol[indx], label = "Sobol",color='red')
            if confidence:
                plt.fill_between(N_row[indx]/ndim, error_lhs_down[indx], error_lhs_up[indx],color = '#539caf',alpha = 0.4,label = '95% CI LHS')
                plt.fill_between(N_row[indx]/ndim, error_rand_down[indx], error_rand_up[indx],color = 'green',alpha = 0.4,label = '95% CI random')
                plt.fill_between(N_row[indx]/ndim, error_sob_down[indx], error_sob_up[indx],color = 'red', alpha = 0.4,   label = '95% CI Sobol')
            plt.yscale('log')
            plt.xlabel('Number of points', fontsize=10)
            plt.ylabel('$\log \epsilon$', rotation=90, labelpad=5)
            plt.legend()
            fnpdf = 'err(rows)_monoms={}.pdf'.format(ndim*i)
            plt.grid(True)
            plt.savefig(fnpdf)
            plt.close(fig)
           
    if rows is not None:
        if type(rows) is not list:
            rows = rows.tolist()
        for row in rows:
            indx = np.where(N_row == row)
            fig = plt.figure()
            plt.plot(ndim*N_col[indx], error[indx],label = 'BMaxvol', linewidth = 3)
            plt.plot(ndim*N_col[indx], error_LHS[indx], label = "LHS",color = '#539caf')
            plt.plot(ndim*N_col[indx], error_random[indx], label = "Random",color = 'green')
            plt.plot(ndim*N_col[indx], error_sobol[indx], label = "Sobol",color = 'red')
            if confidence:
                plt.fill_between(ndim*N_col[indx], error_lhs_down[indx], error_lhs_up[indx],color = '#539caf',alpha = 0.4,label = '95% CI LHS')
                plt.fill_between(ndim*N_col[indx], error_rand_down[indx], error_rand_up[indx],color = 'green',alpha = 0.4,label = '95% CI random')
                plt.fill_between(ndim*N_col[indx], error_sob_down[indx], error_sob_up[indx],  color = 'red',  alpha = 0.4,label = '95% CI Sobol')
            plt.yscale('log')
            plt.xlabel('Number of expansion terms', fontsize=10)
            plt.ylabel('$\log \epsilon$', rotation=90,labelpad=5)
            plt.legend()
            fnpdf = 'err(cols)_row={}.pdf'.format(row)
            plt.grid(True)
            plt.savefig(fnpdf)
            plt.close(fig)    
            
            
            
def only_bmaxvol_error(fn, error, rows):
    ndim = 3
    error = np.array(error)
    N_row, N_col,_ = file_extraction(fn, new_extr=True)
    if type(rows) is not list:
            rows = rows.tolist()
    for row in rows:
        indx = np.where(N_row == row)
        fig = plt.figure()
        plt.plot(ndim*N_col[indx], error[indx],label = 'BMaxvol', linewidth = 3)
        plt.yscale('log')
        plt.xlabel('Number of expansion terms', fontsize=10)
        plt.ylabel('$\log \epsilon$', rotation=90,labelpad=5)
        plt.legend()
        fnpdf = 'err(cols)_row={}.pdf'.format(row)
        plt.grid(True)
        plt.savefig(fnpdf)
        plt.close(fig)    


In [ ]:
only_bmaxvol_error(dir_points + "distrib=" + points_distrib + "_radius=" + rad + ".txt", error_e,[210])

In [ ]:
multi_error_slice(dir_points + "distrib=" + points_distrib + "_radius=" + rad + ".txt", 1.8, error, error_LHS, error_random, error_sobol, log_it=True, adjust = False)

In [ ]:
multi_error_straight(dir_points + "distrib=" + points_distrib + "_radius=" + rad + ".txt", error_e, a,b,c,1, np.arange(54,90,3),confidence=True)

## Separate B-maxvol test

In [ ]:
num_points_for_big_matrix = 1200
cut_radiuses = [0.05]
expansion = 12
row_exp = 30
nder = 2
p_size = expansion*(nder+1)
distr = 'LHS'
x = test_points_gen(num_points_for_big_matrix, nder, distrib=distr)
A = GenMat(p_size+3, x, poly=cheb, debug=False, pow_p=1)
A = matrix_prep(A, nder+1)

In [ ]:
fn_pre_pdf = "distrib={}".format(distr)
for cut_radius in cut_radiuses:
    f = open(os.path.join(dir_str, "distrib={}_radius={}".format(distr, cut_radius) + '.txt'), "w")       
    fnpdf = os.path.join(dir_pdf, fn_pre_pdf + "_expansion={}_N_rows_ex={}.pdf".format(p_size, row_exp))
    try:
        if cut_radius == None:
            to_erase = None
        else:    
            erase_init(point_erase, x, nder, r = cut_radius)
            to_erase = point_erase
        pivs = rect_block_maxvol(A, nder, Kmax = row_exp*(nder+1), max_iters=100, rect_tol = 0.05, tol = 0.0, debug = False, to_erase = to_erase)
    except SingularError as err:
        print ('not full column rank with expansion={}, N_rows_ex={}, err={}'.format(
                                            expansion, row_exp, err.value)) 
        #continue
        break
    cut_piv = pivs[:row_exp*(nder+1)]
    taken_indices = cut_piv[::(nder+1)] // (nder+1)
    

    l_bound = np.amin(x, 0)
    u_bound = np.amax(x, 0)
    delta = (u_bound - l_bound)/20.0
    fig = plt.figure()
    plt.xlim(l_bound[0] - delta[0], u_bound[0] + delta[0])
    plt.ylim(l_bound[1] - delta[1], u_bound[1] + delta[1])
    plt.plot(x[pivs[::(nder+1)] // (nder+1), 0], x[pivs[::(nder+1)] // (nder+1), 1], 'b^')
    plt.plot(x[taken_indices, 0], x[taken_indices, 1], 'r^')
    plt.grid(True)

  
    fnpdf = 'columns={}_rows={}_rad={}.pdf'.format(expansion*(nder+1), row_exp*(nder+1), cut_radius)

    plt.savefig(fnpdf)
    plt.close(fig)
    taken_indices.tofile(f, sep=" ")
    f.write("_Nrows={}_expans={}\n".format(row_exp*(nder+1), expansion))
    f.flush()    

In [ ]:
col_exp = 4
i = np.where(N_col == col_exp)[0][0]
taken_indices = p_indices[i]
def parameter_plot(N_points):
    l_bound = np.amin(x, 0)
    u_bound = np.amax(x, 0)
    delta = (u_bound - l_bound)/20.0
    fig = plt.figure(figsize=(8,8))
    plt.xlim(l_bound[0] - delta[0], u_bound[0] + delta[0])
    plt.ylim(l_bound[1] - delta[1], u_bound[1] + delta[1])
    plt.plot(x[taken_indices[:N_points], 0], x[taken_indices[:N_points], 1], 'r^')
    plane_shape(None)
    plt.grid(True)
    fnpdf = 'Chosen_points_N={}.pdf'.format(N_points)
    plt.savefig(fnpdf)
    
controls = {'N_points': widgets.IntSlider(
    min=col_exp, max=max_row, step=1, value=2, continuous_update=False, description='$Points$')
}

In [ ]:
interact(parameter_plot, **controls);

### Draw areas

In [ ]:
#col_exp = min_expansion
col_exp = 40
i = np.where(N_col == col_exp)[0][0]
taken_indices = p_indices[i]
N_points = len(p_indices[i])

In [ ]:
parameter_plot(len(p_indices[i]))

In [ ]:
def quantum_wave_shape(x=None):
    theta = np.arange(0,2*np.pi, 0.01)
    r = 0.8 + 0.2*np.cos(7*theta)
    xx = r*np.cos(theta)
    yy = r*np.sin(theta)
    plt.figure(figsize=(7,7))
    plt.plot(xx, yy, color = 'blue',lw=2)
    if x is not None:
        plt.plot(x[:, 0], x[:, 1], 'r^')
    plt.grid(True)

In [ ]:
def ellipse_shape(x=None):
    phi = np.arange(0,2*np.pi, 0.01)
    b= 0.2
    e = 0.95
    r = (b / np.sqrt(1-(e*np.cos(phi))**2))
    xx = r*np.cos(phi)
    yy = r*np.sin(phi)
    fig = plt.figure(figsize=(8,6))
    plt.plot(xx, yy, color = 'blue')
    if x is not None:
        plt.plot(x[:, 0], x[:, 1], 'r^')
    fnpdf = 'ellipse.pdf'
    plt.grid(True)
    plt.savefig(fnpdf)    

In [ ]:
def f1(x):
    return(np.sqrt((0.11)**2 - (x+0.89)**2)) 
def f2(x):
    return(0.11)
def f3(x):
    return(1.5*x + 0.56)
def f4(x):
    return(0.95)
def f6(x):
    return(1.1*x - 0.66)
def f7(x):
    return(0.33)
def f8(x):
    return(2*x + 1) 
def f9(x):
    return(-2*x + 1)
f2vec = np.vectorize(f2, signature='()->()')
f4vec = np.vectorize(f4, signature='()->()')
f7vec = np.vectorize(f7, signature='()->()')

In [ ]:
x = complex_area_pnts_gen(1400, nder, mod='plane', distrib='Sobol')

In [ ]:
def plane_shape(x=None):
    x1 = np.linspace(-1,-0.89,200)
    x2 = np.linspace(-0.89,-0.3,200)
    x3 = np.linspace(-0.3,0.26,200)
    x4 = np.linspace(0.26,0.36,200)
    y5 = np.linspace(0.95,0.11,100)
    x5 = np.empty(y5.shape)
    x5.fill(0.36)
    x6 = np.linspace(0.36,0.7,200)
    x7 = np.linspace(0.7,0.9,200)
    x8 = np.linspace(0.9,0.95,200)
    y9  = np.linspace(0.33,-0.33,100)
    x9 = np.empty(y9.shape)
    x9.fill(0.95)
    fig = plt.figure(figsize=(8,8))
    plt.plot(x1,f1(x1), color="blue",lw=2)
    plt.plot(x1,-1*f1(x1), color="blue",lw=2)
    plt.plot(x2,f2vec(x2), color="blue",lw=2)
    plt.plot(x2,-1*f2vec(x2), color="blue",lw=2)
    plt.plot(x3,f3(x3), color="blue",lw=2)
    plt.plot(x3,-1*f3(x3), color="blue",lw=2)
    plt.plot(x4,f4vec(x4), color="blue",lw=2)
    plt.plot(x4,-1*f4vec(x4), color="blue",lw=2)
    plt.plot(x5, y5, color = 'blue',lw=2)
    plt.plot(x5, -y5[::-1], color = 'blue',lw=2)
    plt.plot(x6,f2vec(x6), color="blue",lw=2)
    plt.plot(x6,-1*f2vec(x6), color="blue",lw=2)
    plt.plot(x7,f6(x7), color="blue",lw=2)
    plt.plot(x7,-1*f6(x7), color="blue",lw=2)
    plt.plot(x8,f7vec(x8), color="blue",lw=2)
    plt.plot(x8,-1*f7vec(x8), color="blue",lw=2)
    plt.plot(x9, y9, color = 'blue',lw=2)
    if x is not None:
        plt.plot(x[:, 0], x[:, 1], 'r^')
    plt.grid(True)

In [ ]:
plane_shape(x1[taken_indices])

In [ ]:
plane_shape(x)

In [ ]:
### Rhombus
x = np.linspace(-1,1,200)
l_bound = np.amin(x, 0)
u_bound = np.amax(x, 0)
delta = (u_bound - l_bound)/20.0

fig = plt.figure(figsize=(7,7))
plt.xlim(l_bound, u_bound)
plt.ylim(l_bound, u_bound)
plt.plot(x[50:100],f8(x[50:100]), color="blue")
plt.plot(x[50:100],-1*f8(x[50:100]), color="blue")
plt.plot(x[100:150],f9(x[100:150]), color="blue")
plt.plot(x[100:150],-1*f9(x[100:150]), color="blue")
plt.plot(x1[taken_indices[:N_points], 0], x1[taken_indices[:N_points], 1], 'r^')
plt.grid(True)

In [ ]:
x = test_points_gen(45000,2)

In [ ]:
x = domain_erase(x,mod='ellipse')

In [ ]:
plt.plot(x[:, 0], x[:, 1], 'r^')

In [ ]:
col_exp = 9
i = np.where(N_col == col_exp)[0][0]
taken_indices = p_indices[i]

In [ ]:
ellipse_shape(x[taken_indices])

In [ ]:
def MakeValsAndNorms(funcs, pnts):
    res = []
    for f in funcs:
        vals = f(*pnts.T)
        norm = np.linalg.norm(vals, np.inf)
        res.append((f, vals, norm))

    return res

# @jit
def LebesgueConst(pnts, l, poly=cheb, test_pnts=None, pow_p=1, funcs=None, derivative = True):
    A = GenMat(l, pnts, poly=poly, debug=False, pow_p=pow_p, ToGenDiff=derivative)
    
    nder = pnts.shape[1]
    if derivative:
        A = matrix_prep(A, nder+1)
    if test_pnts is None:
        test_pnts = test_points_gen(int(1e5), nder)

    ABig = GenMat(l, test_pnts, poly=poly, debug=False, pow_p=pow_p, ToGenDiff=False)
    # F = A.dot(np.linalg.solve(A.T.dot(A), ABig.T)) # Slower
    F = np.linalg.pinv(A).T.dot(ABig.T)

    maxx = np.max(np.sum(np.abs(F), axis=0))
    if funcs is not None:
        res = np.empty(1 + len(funcs))
        res[0] = maxx
        rs = 0
        for f, fvals, fnorm in funcs:
            rs += 1
            rhs = RHS(f, pnts, derivative = derivative)
            res[rs] = np.linalg.norm(F.T.dot(rhs) - fvals, np.inf)/fnorm

        return res
    else:
        return  maxx

In [ ]:
qwe = np.array(b)

In [ ]:
qwe.reshape(len(b),1)

In [ ]:
ValsandNorms = MakeValsAndNorms(functions, points_test)

In [ ]:
t,y = LebesgueConst(x1[taken_indices], 12, poly=cheb, test_pnts=points_test, pow_p=1, funcs=ValsandNorms, derivative = True)

In [ ]:
def LebesgueConst_w(pnts, l,uwu, poly=cheb,test_pnts=None, pow_p=1, funcs=None):
    A = GenMat(l, pnts, poly=poly, debug=False, pow_p=pow_p, ToGenDiff=True)
    nder = pnts.shape[1]
    A = matrix_prep(A, nder+1)
    ewe = np.array([[1],[1],[1]])
    weeght = np.kron(uwu,ewe)
    C = weeght*A
    ABig = GenMat(l, test_pnts, poly=poly, debug=False, pow_p=pow_p, ToGenDiff=False)
    F = np.linalg.pinv(A).T.dot(ABig.T)
    for f, fvals, fnorm in funcs:
        rhs = RHS(f, pnts, derivative = True)
        print rhs.shape
        print weeght.shape
        rhs1 = weeght.T*rhs
        rhs1 = rhs1.reshape(rhs1.shape[1])
        #rhs = rhs1.reshape(42)
        res = np.linalg.norm(F.T.dot(rhs) - fvals, np.inf)/fnorm

    return res


In [ ]:
A = GenMat(6, x, poly=power, debug=False, pow_p=1)

In [ ]:
taken_points = np.load("piston_10_error.npz")

In [ ]:
x = taken_points['er']

In [ ]:
data = np.load("piston_10.npz")

In [ ]:
er_lhs = data['er_lhs']
er_r = data['er_r']
er_s = data['er_s']

In [ ]:
def multi_error_piston(error, err_LHS, err_rand, err_sob):
    ndim = 8
    k = 0.6
    error = np.array(error)
    error_LHS = np.mean(err_LHS, axis = 1) 
    error_random = np.mean(err_rand,axis = 1)
    error_sobol = np.mean(err_sob,axis = 1)

    error_lhs_up = k*np.std(err_LHS,axis=1) + error_LHS
    error_lhs_down = error_LHS - k*np.std(err_LHS,axis=1)

    error_rand_up = k*np.std(err_rand,axis=1) + error_random
    error_rand_down = error_random - k*np.std(err_rand,axis=1)

    error_sob_up = k*np.std(err_sob,axis=1) + error_sobol
    error_sob_down = error_sobol - k*np.std(err_sob,axis=1)
  

    indx = np.arange(0,10)
    N_row = np.arange(1600,800,-80)
    fig = plt.figure()
    plt.plot(N_row[indx]/ndim, error[indx], label = 'BMaxvol', linewidth= 3)
    plt.plot(N_row[indx]/ndim, error_LHS[indx], label = "LHS",color = '#539caf')
    plt.plot(N_row[indx]/ndim, error_random[indx], label = "Random",color = 'green')
    plt.plot(N_row[indx]/ndim, error_sobol[indx], label = "Sobol",color='red')
    
    
    plt.fill_between(N_row[indx]/ndim, error_lhs_down[indx], error_lhs_up[indx],color = '#539caf',alpha = 0.4,label = '95% CI LHS')
    plt.fill_between(N_row[indx]/ndim, error_rand_down[indx], error_rand_up[indx],color = 'green',alpha = 0.4,label = '95% CI random')
    plt.fill_between(N_row[indx]/ndim, error_sob_down[indx], error_sob_up[indx],color = 'red', alpha = 0.4,   label = '95% CI Sobol')
    
    plt.yscale('log')
    plt.xlabel('Number of points', fontsize=10)
    plt.ylabel('$\log \epsilon$', rotation=90, labelpad=5)
    plt.legend()
    fnpdf = 'err(rows)_monoms={}.pdf'.format(ndim*100)
    plt.grid(True)
    plt.savefig(fnpdf)
    plt.close(fig)

In [ ]:
multi_error_piston(x,er_lhs, er_r, er_s)

### Radial functions

In [ ]:
def ellipse_sp(x,y):
    b,e = 0.2,0.95
    sigma,n = 1/2, 1
    r = sp.sqrt(x**2 + y**2)
    phi = sp.atan2(y,x)
    R = r*sp.sqrt(1 - (e*sp.cos(phi))**2)/b
    return((sp.exp(-1*(R**2)/(2*(sigma**2))))*sp.cos(n*phi))

def sin_blob_sp(x,y):
    a = 0.2
    b = 0.8
    sigma,n = 5,1
    r = sp.sqrt(x**2 + y**2)
    phi = sp.atan2(y,x)
    R = r*(1/(b + a*sp.cos(n*phi)))
    return((sp.exp(-1*(R**2)/(2*(sigma**2))))*sp.cos(n*phi))


f_ellipse = symb_to_func(ellipse_sp,    2, True, False, name='Ellipse')  
f_sin_blob= symb_to_func(sin_blob_sp,   2, True, False, name='Blob')  

f_ellipse.diff    = MakeDiffs(ellipse_sp,  2)
f_sin_blob.diff   = MakeDiffs(sin_blob_sp, 2)

In [ ]:
fig = plt.figure(figsize=(7,7))
ax = fig.gca(projection='3d')

# Make data.
a = 1.1
X = np.linspace(-a, a, 1000)
Y = np.linspace(-a, a, 1000)
phi = np.arange(0,2*np.pi, 0.01)
b= 0.2
e = 0.95
X, Y = np.meshgrid(X, Y)

#r = (b / np.sqrt(1-(e*np.cos(phi))**2))
#xx = r*np.cos(phi)
#yy = r*np.sin(phi)

r = 0.8 + 0.2*np.cos(7*phi)
xx = r*np.cos(phi)
yy = r*np.sin(phi)

#Z = f_ellipse(X,Y)
Z = f_sin_blob(X,Y)

# Plot the surface.
surf = ax.plot_surface(X, Y, Z, cmap=cm.summer,
                       linewidth=0,label = 'Trigonometric')
fc = ax.plot(xx, yy,'bo', zs=0, zdir='z', label='curve in (x,y)')
# Customize the z axis.
ax.set_zlim(-1, 1)
# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.4, aspect=5)
plt.grid(True)
#fnpdf = 'ellips.pdf'
#plt.savefig(fnpdf)
plt.show()

In [ ]:
### BMaxvol and maxvol
x = complex_area_pnts_gen(num_points_for_big_matrix, nder, mod=domain_type, distrib='LHS')
function = [f_sin_blob]
n_points_start = 20
n_points_end = 16
step = 2
error_block = []
error_nonblock = []
if complex_domain:
    x = domain_erase(x, mod = domain_type)
    
for n_points in range(n_points_start, n_points_end, step):    
    print(n_points,)
    A = GenMat(int((nder+1)*n_points/2), x, poly=cheb, debug=False, pow_p=1)
    A = matrix_prep(A, nder+1)
    B = GenMat(int(n_points/2),x,poly=cheb,debug=False, pow_p=1)
    B = B[:x.shape[0]]
    
    N_rows = n_points*(nder+1)

    try:
        erase_init(point_erase, x, nder, r = cut_radius)
        to_erase = point_erase
        pivs = rect_block_maxvol(A, nder, Kmax = N_rows, max_iters=100, rect_tol = 0.05, tol = 0.0, debug = False, to_erase = to_erase)
    except SingularError as err:
        print ('not full column rank with err={}'.format(err.value)) 
        continue

    cut_piv = pivs[:N_rows]
    taken_points_block = cut_piv[::(nder+1)] // (nder+1)    
    taken_points_nonblock,_ = rect_maxvol(B, maxK = n_points, minK = n_points)
    
    ValsandNorms = MakeValsAndNorms(function, points_test)
    _,er_bmv = LebesgueConst(x[taken_points_block], int((nder+1)*n_points/2), poly=cheb, test_pnts=points_test, pow_p=1, funcs=ValsandNorms, derivative = True)
    error_block.append(er_bmv)
    _,er_mv = LebesgueConst(x[taken_points_nonblock], int(n_points/2), poly=cheb, test_pnts=points_test, pow_p=1, funcs=ValsandNorms, derivative = False)
    error_nonblock.append(er_mv)

In [ ]:
### just maxvol (and lhs)
x = complex_area_pnts_gen(num_points_for_big_matrix, nder, mod = domain_type, distrib='LHS')
x_test = complex_area_pnts_gen(n_test, nder, mod = domain_type, distrib='LHS')
function = [f_sin_blob]
n_points_start = 6
n_points_end = 200
step = 2
error_nonblock = []
error_lhs = []
    
for n_points in range(n_points_start, n_points_end, step):    
    B = GenMat(int(n_points/2),x,poly=cheb,debug=False, pow_p=1)
    B = B[:x.shape[0]]

    taken_points_nonblock,_ = rect_maxvol(B, maxK = n_points, minK = n_points)
    
    ValsandNorms = MakeValsAndNorms(function, x_test)
    
    _, er_mv = LebesgueConst(x[taken_points_nonblock], int(n_points/2), poly=cheb, test_pnts=x_test, pow_p=1, funcs=ValsandNorms, derivative = False)
    _,er_lhs = LebesgueConst(test_points_gen(n_points, nder, distrib='LHS'), int(n_points/2), poly=cheb, test_pnts=x_test, pow_p=1, funcs=ValsandNorms, derivative = False)
    
    error_nonblock.append(er_mv)
    error_lhs.append(er_lhs)

In [ ]:
fig = plt.figure()
plt.plot(np.arange(n_points_start, n_points_end, step), error_lhs,label = 'lhs', linewidth = 3)
plt.plot(np.arange(n_points_start, n_points_end, step), error_nonblock,label = 'maxvol', linewidth = 2)
plt.yscale('log')
plt.xlabel('Number of points', fontsize=10)
plt.ylabel('$\log \epsilon$', rotation=90,labelpad=5)
plt.legend()
fnpdf = 'err(points).pdf'
plt.grid(True)
plt.savefig(fnpdf)
plt.close(fig) 